**Ctrl**+Shift+i

```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```



In [1]:
!nvidia-smi -L  

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!pip install -qqq datasets folium==0.9.1 rouge-score==0.0.4 nltk wandb 

     |████████████████████████████████| 365 kB 5.1 MB/s 
     |████████████████████████████████| 91 kB 7.9 MB/s 
     |████████████████████████████████| 1.8 MB 54.8 MB/s 
     |████████████████████████████████| 115 kB 48.3 MB/s 
     |████████████████████████████████| 141 kB 45.2 MB/s 
     |████████████████████████████████| 212 kB 37.7 MB/s 
     |████████████████████████████████| 101 kB 5.0 MB/s 
     |████████████████████████████████| 596 kB 36.5 MB/s 
     |████████████████████████████████| 127 kB 39.6 MB/s 
     |████████████████████████████████| 181 kB 36.1 MB/s 
     |████████████████████████████████| 157 kB 32.8 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 157 kB 35.8 MB/s 
     |████████████████████████████████| 157 kB 39.9 MB/s 
     |████████████████████████████████| 157 kB 35.6 MB/s 
     |████████████████████████████████| 157 kB 35.6 MB/s 
     |████████████████████████████████| 156 kB 39.6 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("cnn_dailymail", '3.0.0')
metric = load_metric("rouge")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
raw_datasets.num_rows

{'train': 287113, 'validation': 13368, 'test': 11490}

In [7]:
raw_datasets = raw_datasets.rename_column("article", "document")
raw_datasets = raw_datasets.rename_column("highlights", "summary")

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [8]:
! pip install -qqq transformers[sentencepiece]==4.20.1

     |████████████████████████████████| 4.4 MB 5.5 MB/s 
     |████████████████████████████████| 6.6 MB 38.4 MB/s 
     |████████████████████████████████| 1.3 MB 30.8 MB/s 


In [9]:
import nltk
nltk.download('punkt')
import transformers

print(transformers.__version__)

4.20.1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
from transformers import AutoTokenizer
initial_model = 'google/t5-v1_1-base'

max_input_length = 512
max_target_length = 128

tokenizer = AutoTokenizer.from_pretrained(initial_model, model_max_length=max_input_length) 

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/605 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [11]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, 
                             max_length=max_input_length, 
                             truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], 
                           max_length=max_target_length, 
                           truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [12]:
#preprocess_function(raw_datasets['train'][2:4])

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [13]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/288 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [20]:
tokenized_datasets = tokenized_datasets.remove_columns(['document', 'summary', 'id'])

In [14]:
from datasets import load_from_disk

In [22]:
tokenized_datasets.save_to_disk('/content/drive/Shareddrives/UCM_SHARED/TFM_ESG/Preprocessed Datasets for training/T5v1_1_CNNDailyFull')

In [16]:
tokenized_datasets.num_rows 

{'train': 287113, 'validation': 13368, 'test': 11490}

In [21]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

In [24]:
# Load preprocessed Data 
tokenized_datasets2 = load_from_disk(dataset_path='/content/drive/Shareddrives/UCM_SHARED/TFM_ESG/Preprocessed Datasets for training/T5v1_1_CNNDailyFull')
tokenized_datasets2.num_rows 

{'train': 287113, 'validation': 13368, 'test': 11490}

In [25]:
tokenized_datasets2

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})